In [1]:
import tempfile
import zipfile
import datetime
import json
import pathlib
import operator
import collections

import pandas as pd

from google_fetch import config


In [2]:
def read_bundle(bundle_path):
    fetch_datetime = datetime.datetime.strptime(
        bundle_path.stem,
        'google-political-ads-transparency-bundle_%Y-%m-%d_%H-%M-%S',
    )

    with tempfile.TemporaryDirectory() as tmpdirname:
        tmp_dir = pathlib.Path(tmpdirname)

        with zipfile.ZipFile(bundle_path, "r") as zip_ref:
            zip_ref.extractall(tmpdirname)

            assert set([
                element.name
                for element in tmp_dir.iterdir()
            ]) == {'google-political-ads-transparency-bundle'}

            data_dir = tmp_dir / 'google-political-ads-transparency-bundle'
            assert set([
                element.name
                for element in data_dir.iterdir()
            ]) == {
                'README.txt',
                'google-political-ads-advertiser-stats.csv',
                'google-political-ads-advertiser-weekly-spend.csv',
                'google-political-ads-campaign-targeting.csv',
                'google-political-ads-creative-stats.csv',
                'google-political-ads-geo-spend.csv',
                'google-political-ads-top-keywords-history.csv',
            }

            bundle_data = {}

            with open(data_dir / 'README.txt', 'r') as f:
                bundle_data['README.txt'] = f.read()

            for key in [
                'google-political-ads-advertiser-stats.csv',
                'google-political-ads-advertiser-weekly-spend.csv',
                'google-political-ads-campaign-targeting.csv',
                'google-political-ads-creative-stats.csv',
                'google-political-ads-geo-spend.csv',
                'google-political-ads-top-keywords-history.csv',
            ]:
                bundle_data[key] = pd.read_csv(
                    data_dir / key,
                    dtype=str,
                    keep_default_na=False,
                    na_values={},
                )

    return fetch_datetime, bundle_data

In [3]:
for bundle_path in sorted(list((config.DATA_DIR / 'google').iterdir())):
    print(bundle_path)

/home/michel/projects/desinfo/political-ads-scraper/data/google/1
/home/michel/projects/desinfo/political-ads-scraper/data/google/2
/home/michel/projects/desinfo/political-ads-scraper/data/google/3
/home/michel/projects/desinfo/political-ads-scraper/data/google/google-political-ads-transparency-bundle_2019-06-14_14-32-03.zip
/home/michel/projects/desinfo/political-ads-scraper/data/google/google-political-ads-transparency-bundle_2019-06-18_14-18-01.zip
/home/michel/projects/desinfo/political-ads-scraper/data/google/google-political-ads-transparency-bundle_2019-06-23_14-18-01.zip


In [30]:
bundle_path_1 = pathlib.Path('/home/michel/projects/desinfo/political-ads-scraper/data/google/google-political-ads-transparency-bundle_2019-06-18_14-18-01.zip')
bundle_path_2 = pathlib.Path('/home/michel/projects/desinfo/political-ads-scraper/data/google/google-political-ads-transparency-bundle_2019-06-23_14-18-01.zip')
fetch_datetime_1, bundle_data_1 = read_bundle(bundle_path_1)
fetch_datetime_2, bundle_data_2 = read_bundle(bundle_path_2)


In [5]:
bundle_data_1['README.txt'] == bundle_data_2['README.txt']

True

## Ads

In [6]:
key = 'google-political-ads-creative-stats.csv'
df_1 = bundle_data_1[key]
df_2 = bundle_data_2[key]

assert list(df_1.columns) == list(df_2.columns)

# A few ads were run by distinct advertisers, for instance:
# https://transparencyreport.google.com/political-ads/advertiser/AR185084640594231296/creative/CR275167628257918976
# https://transparencyreport.google.com/political-ads/advertiser/AR242125929453715456/creative/CR275167628257918976
duplicate_ad_ids_1 = set([
    ad_id
    for ad_id, count in collections.Counter(df_1['Ad_ID']).items()
    if count > 1
])
duplicate_ad_ids_2 = set([
    ad_id
    for ad_id, count in collections.Counter(df_2['Ad_ID']).items()
    if count > 1
])


common_duplicates = duplicate_ad_ids_1 & duplicate_ad_ids_2
old_duplicates = duplicate_ad_ids_1 - duplicate_ad_ids_2
new_duplicates = duplicate_ad_ids_2 - duplicate_ad_ids_1

print('{} were run by several advertisers (eg {})'.format(len(duplicate_ad_ids_1), list(duplicate_ad_ids_1)[0]))
print('{} are now run by several advertisers (eg {})'.format(len(duplicate_ad_ids_2), list(duplicate_ad_ids_2)[0]))


367 were run by several advertisers (eg CR109616361486942208)
360 are now run by several advertisers (eg CR109616361486942208)


In [7]:

indexes = ['Ad_ID', 'Advertiser_ID']
df_1.set_index(indexes, inplace=True)
df_2.set_index(indexes, inplace=True)

ids_1 = set(df_1.index)
ids_2 = set(df_2.index)

assert len(ids_1) == len(df_1)
assert len(ids_2) == len(df_2)

removed_ids = ids_1 - ids_2
added_ids = ids_2 - ids_1
common_ids = ids_1 & ids_2
assert len(ids_1) + len(added_ids) - len(removed_ids) == len(ids_2)
assert len(ids_1 | ids_2) == len(common_ids) + len(added_ids) + len(removed_ids)

print('{} ads were removed, {} were added'.format(len(removed_ids), len(added_ids)))


279 ads were removed, 1339 were added


In [8]:
removed_ads = []
moved_ads = []

for ad_id, adv_id in removed_ids:
    try:
        new_advertisers = list(df_2.loc[[ad_id]].reset_index()['Advertiser_ID'])
    except KeyError:
        removed_ads.append((ad_id, adv_id))
    else:
        if len(new_advertisers) == 1:
            moved_ads.append((ad_id, adv_id, new_advertisers[0]))
        else:
            print(new_advertisers)

print('{} ads were removed. A few examples:\n'.format(len(removed_ads)))
for ad_id, adv_id in removed_ads[:3]:
    print('{}\n    by {}\n    was removed.\n'.format(
        'https://transparencyreport.google.com/political-ads/advertiser/{}/creative/{}'.format(adv_id, ad_id),
        'https://transparencyreport.google.com/political-ads/advertiser/{}'.format(adv_id),
    ))

print('{} ads changed its advertiser. A few examples:\n'.format(len(moved_ads)))
for ad_id, adv_id, new_adv_id in moved_ads[:3]:
    print('{}\n    by {}\n    was replaced by\n{}\n    by {}\n'.format(
        'https://transparencyreport.google.com/political-ads/advertiser/{}/creative/{}'.format(adv_id, ad_id),
        'https://transparencyreport.google.com/political-ads/advertiser/{}'.format(adv_id),
        'https://transparencyreport.google.com/political-ads/advertiser/{}/creative/{}'.format(new_adv_id, ad_id),
        'https://transparencyreport.google.com/political-ads/advertiser/{}'.format(new_adv_id),
    ))


214 ads were removed. A few examples:

https://transparencyreport.google.com/political-ads/advertiser/AR242125929453715456/creative/CR179596703182094336
    by https://transparencyreport.google.com/political-ads/advertiser/AR242125929453715456
    was removed.

https://transparencyreport.google.com/political-ads/advertiser/AR488306308034854912/creative/CR117534219596464128
    by https://transparencyreport.google.com/political-ads/advertiser/AR488306308034854912
    was removed.

https://transparencyreport.google.com/political-ads/advertiser/AR89517838687010816/creative/CR161477026434252800
    by https://transparencyreport.google.com/political-ads/advertiser/AR89517838687010816
    was removed.

65 ads changed its advertiser. A few examples:

https://transparencyreport.google.com/political-ads/advertiser/AR322201712037199872/creative/CR517478000787193856
    by https://transparencyreport.google.com/political-ads/advertiser/AR322201712037199872
    was replaced by
https://transparencyr

In [9]:
adv_with_removed_ad = set([
    adv_id
    for _, adv_id in removed_ads
])
print(adv_with_removed_ad)

{'AR98551701098725376', 'AR144491221052555264', 'AR374597289391423488', 'AR80178586920681472', 'AR166715924462698496', 'AR140403236820484096', 'AR507835283811598336', 'AR230475229367894016', 'AR210958348219056128', 'AR180201434577371136', 'AR488306308034854912', 'AR419626413717454848', 'AR50338116220747776', 'AR334064892745089024', 'AR94030784163217408', 'AR316331969212317696', 'AR77689292595396608', 'AR89517838687010816', 'AR503512828724903936', 'AR390238117174444032', 'AR400032566754672640', 'AR17917091730423808', 'AR495298652231696384', 'AR60585289713713152', 'AR185084640594231296', 'AR485896728302583808', 'AR119011138590474240', 'AR288284801977286656', 'AR242125929453715456'}


## Advertisers

In [31]:
key = 'google-political-ads-advertiser-stats.csv'
df_1 = bundle_data_1[key]
df_2 = bundle_data_2[key]

assert list(df_1.columns) == list(df_2.columns)

indexes = 'Advertiser_ID'
df_1.set_index(indexes, inplace=True)
df_2.set_index(indexes, inplace=True)
ids_1 = set(df_1.index)
ids_2 = set(df_2.index)

assert len(ids_1) == len(df_1)
assert len(ids_2) == len(df_2)

removed_ids = ids_1 - ids_2
added_ids = ids_2 - ids_1
common_ids = ids_1 & ids_2
assert len(ids_1) + len(added_ids) - len(removed_ids) == len(ids_2)
assert len(ids_1 | ids_2) == len(common_ids) + len(added_ids) + len(removed_ids)

print('{} advertisers were removed, {} were added'.format(len(removed_ids), len(added_ids)))


12 advertisers were removed, 25 were added


In [25]:
df_adv_with_removed_ad = df_2.loc[adv_with_removed_ad]
ad_adv_eu = df_adv_with_removed_ad[df_adv_with_removed_ad['Regions'].map(lambda x: 'EU' in x)]
ad_adv_eu

,Advertiser_Name,Public_IDs_List,Regions,Elections,Total_Creatives,Spend_USD,Spend_EUR,Spend_INR,Spend_BGN,Spend_HRK,Spend_CZK,Spend_DKK,Spend_HUF,Spend_PLN,Spend_RON,Spend_SEK,Spend_GBP
Advertiser_ID,,,,,,,,,,,,,,,,,
AR374597289391423488,"KKW Lewica Razem – Razem, Unia Pracy, RSS",,"EU, PL",EU-Parliament,51,4300,3750,302000,7425,28200,97000,28500,1215000,16300,18000,39750,3250
AR166715924462698496,Asociația Platforma Inițiativa România,,"EU, RO",EU-Parliament,8,0,50,3250,75,300,1000,500,15000,200,225,750,50
AR210958348219056128,Kansallinen Kokoomus r.p.,,"EE, ES, EU, FI, FR, SE",EU-Parliament,193,101600,89400,7205500,176825,671700,2325000,674500,28927500,388200,429825,942000,77250
AR50338116220747776,Bündnis 90/Die Grünen Landesverband RLP,,"DE, EU",EU-Parliament,174,4000,3550,284500,6975,26500,92000,26500,1140000,15300,16950,37500,3050
AR316331969212317696,Konfederacja KORWiN Braun Liroy Narodowcy,,"EU, PL",EU-Parliament,15,46600,41000,3303500,81075,308000,1066000,309500,13260000,178000,197100,432000,35400
AR400032566754672640,Fratelli d'Italia Centrodestra Nazionale,,"EU, IT",EU-Parliament,271,61700,54300,4374250,107350,407800,1411000,409500,17557500,235700,260925,571500,46900
AR17917091730423808,Bündnis 90/Die Grünen Baden-Württemberg,,"DE, EU",EU-Parliament,150,4800,4200,338500,8300,31600,109000,31500,1357500,18200,20175,44250,3650
AR495298652231696384,EVOLVEDREAMSOFT SRL,,"ES, EU, GB, IT, RO",EU-Parliament,528,21100,18550,1494250,36675,139300,482000,140000,6000000,80500,89175,195000,16000
AR185084640594231296,Sloboda a Solidarita ( SaS ),,"AT, BE, EU, HU, SK",EU-Parliament,936,12500,11000,884750,21700,82500,285000,83000,3555000,47700,52800,115500,9500


In [32]:
changed_weekly_spends = ['AR132631338879549440', 'AR488306308034854912', 'AR98551701098725376', 'AR119011138590474240', 'AR486291452976955392', 'AR119011138590474240', 'AR81539232560054272', 'AR419626413717454848', 'AR383080296477622272', 'AR55967615754960896', 'AR400032566754672640', 'AR201649882778304512', 'AR519831505426448384', 'AR474748779908562944', 'AR77689292595396608', 'AR81539232560054272', 'AR140403236820484096', 'AR6195198266703872', 'AR240608328529477632', 'AR488306308034854912', 'AR383251270535741440', 'AR488306308034854912', 'AR399188966458261504', 'AR485896728302583808', 'AR288284801977286656', 'AR140403236820484096', 'AR474748779908562944', 'AR98551701098725376', 'AR73057599863390208', 'AR227104401595039744', 'AR288179248861020160', 'AR94030784163217408', 'AR373464517536907264', 'AR507019171305881600', 'AR227104401595039744', 'AR94030784163217408', 'AR383251270535741440', 'AR6195198266703872', 'AR488306308034854912', 'AR390238117174444032', 'AR362903983230025728', 'AR488306308034854912', 'AR485896728302583808', 'AR185084640594231296', 'AR519831505426448384', 'AR240608328529477632', 'AR140403236820484096', 'AR352083414423175168', 'AR89517838687010816', 'AR120847323008860160', 'AR375716317350592512', 'AR60189740405620736', 'AR373464517536907264', 'AR89517838687010816', 'AR144491221052555264', 'AR390238117174444032', 'AR191840040035287040', 'AR489541609348661248', 'AR140403236820484096', 'AR210958348219056128', 'AR94030784163217408', 'AR40198145111490560', 'AR383251270535741440', 'AR375716317350592512', 'AR210958348219056128', 'AR522320249995919360', 'AR121262938404159488', 'AR94030784163217408', 'AR214800866480226304', 'AR334064892745089024', 'AR89517838687010816', 'AR216489716340490240', 'AR242125929453715456', 'AR485896728302583808', 'AR214800866480226304', 'AR144491221052555264', 'AR214800866480226304', 'AR383251270535741440', 'AR383251270535741440', 'AR400032566754672640', 'AR132631338879549440', 'AR288179248861020160', 'AR390238117174444032', 'AR119011138590474240', 'AR191840040035287040', 'AR166715924462698496', 'AR200379397092409344', 'AR315375668974059520', 'AR488306308034854912', 'AR77689292595396608', 'AR140403236820484096', 'AR488306308034854912', 'AR94030784163217408', 'AR140403236820484096', 'AR180201434577371136', 'AR17917091730423808', 'AR478461555797655552', 'AR6195198266703872', 'AR485896728302583808', 'AR432363981047332864', 'AR400032566754672640', 'AR362903983230025728', 'AR334064892745089024', 'AR60585289713713152', 'AR119011138590474240', 'AR140403236820484096', 'AR50338116220747776', 'AR210958348219056128', 'AR94030784163217408', 'AR530356305605427200', 'AR94030784163217408', 'AR240608328529477632', 'AR383251270535741440', 'AR383251270535741440', 'AR210958348219056128', 'AR40198145111490560', 'AR132631338879549440', 'AR94030784163217408', 'AR175306133932605440', 'AR40198145111490560', 'AR474748779908562944', 'AR77689292595396608', 'AR94030784163217408', 'AR383251270535741440', 'AR240608328529477632', 'AR383251270535741440', 'AR217815727363588096', 'AR503512828724903936', 'AR495298652231696384', 'AR6195198266703872', 'AR288179248861020160', 'AR419626413717454848', 'AR98551701098725376', 'AR428206177826897920', 'AR522320249995919360', 'AR120847323008860160', 'AR240608328529477632', 'AR478461555797655552', 'AR488306308034854912', 'AR32708546780987392', 'AR240608328529477632', 'AR152882144039927808', 'AR488306308034854912', 'AR488306308034854912', 'AR205658702173175808', 'AR17917091730423808', 'AR519831505426448384', 'AR240608328529477632', 'AR144491221052555264', 'AR240608328529477632', 'AR144491221052555264', 'AR383251270535741440', 'AR519831505426448384', 'AR6195198266703872', 'AR362903983230025728', 'AR400032566754672640', 'AR144491221052555264', 'AR217815727363588096', 'AR488306308034854912', 'AR400032566754672640', 'AR488306308034854912', 'AR100996740080992256', 'AR474748779908562944', 'AR488306308034854912', 'AR485896728302583808', 'AR230475229367894016', 'AR80178586920681472', 'AR288179248861020160', 'AR75218140211970048', 'AR488306308034854912', 'AR562822410072489984', 'AR198061076825243648', 'AR119011138590474240', 'AR311843487869829120', 'AR532605081762136064', 'AR530356305605427200', 'AR6195198266703872', 'AR77689292595396608', 'AR352083414423175168', 'AR120847323008860160', 'AR157291735423123456', 'AR316331969212317696', 'AR474748779908562944', 'AR6195198266703872', 'AR50710026028843008', 'AR503512828724903936', 'AR50338116220747776', 'AR13000625486823424', 'AR201649882778304512', 'AR140403236820484096', 'AR132631338879549440', 'AR94030784163217408', 'AR220642022002786304', 'AR89517838687010816', 'AR383251270535741440', 'AR352083414423175168', 'AR240608328529477632', 'AR77689292595396608', 'AR503512828724903936', 'AR300281023592136704', 'AR383251270535741440', 'AR81539232560054272', 'AR242125929453715456', 'AR214800866480226304', 'AR485896728302583808', 'AR230475229367894016', 'AR373464517536907264', 'AR376667944664432640', 'AR140403236820484096', 'AR77689292595396608', 'AR352083414423175168', 'AR315375668974059520', 'AR98551701098725376', 'AR475747136466583552', 'AR227104401595039744', 'AR50338116220747776', 'AR89517838687010816', 'AR17917091730423808', 'AR240608328529477632', 'AR334064892745089024', 'AR77689292595396608', 'AR288179248861020160', 'AR350209296853630976', 'AR374597289391423488', 'AR419626413717454848', 'AR6195198266703872', 'AR513786390496935936', 'AR140403236820484096', 'AR519831505426448384', 'AR374597289391423488', 'AR185084640594231296', 'AR488306308034854912', 'AR503512828724903936', 'AR98551701098725376', 'AR383251270535741440', 'AR350471805254762496', 'AR352083414423175168', 'AR383251270535741440', 'AR237678679797268480', 'AR94030784163217408', 'AR230475229367894016', 'AR237340030215913472', 'AR94030784163217408', 'AR474748779908562944', 'AR488306308034854912', 'AR334064892745089024', 'AR144491221052555264', 'AR242125929453715456', 'AR374590417443749888', 'AR94030784163217408', 'AR40198145111490560', 'AR240608328529477632', 'AR288179248861020160', 'AR144491221052555264', 'AR488306308034854912', 'AR428206177826897920', 'AR210958348219056128', 'AR334064892745089024', 'AR17917091730423808', 'AR271193993235136512', 'AR288179248861020160', 'AR89517838687010816', 'AR180201434577371136', 'AR140403236820484096', 'AR98551701098725376', 'AR283015942256984064', 'AR240608328529477632', 'AR80178586920681472', 'AR89517838687010816', 'AR17917091730423808', 'AR119011138590474240', 'AR488306308034854912', 'AR399188966458261504', 'AR73057599863390208', 'AR98551701098725376', 'AR194446432348930048', 'AR400032566754672640', 'AR489541609348661248', 'AR486291452976955392', 'AR485896728302583808', 'AR119011138590474240', 'AR98551701098725376', 'AR6195198266703872', 'AR194446432348930048', 'AR6195198266703872', 'AR288179248861020160', 'AR489541609348661248', 'AR94030784163217408', 'AR98551701098725376', 'AR140403236820484096', 'AR428206177826897920', 'AR183859784640888832', 'AR383251270535741440']
df_adv_with_removed_ad = df_2.loc[changed_weekly_spends]
ad_adv_eu = df_adv_with_removed_ad[df_adv_with_removed_ad['Regions'].map(lambda x: 'EU' in x)]
ad_adv_eu

,Advertiser_Name,Public_IDs_List,Regions,Elections,Total_Creatives,Spend_USD,Spend_EUR,Spend_INR,Spend_BGN,Spend_HRK,Spend_CZK,Spend_DKK,Spend_HUF,Spend_PLN,Spend_RON,Spend_SEK,Spend_GBP
Advertiser_ID,,,,,,,,,,,,,,,,,
AR132631338879549440,Lietuvos Respublikos liberalų sąjūdis,,"BG, EU, LT",EU-Parliament,264,8800,7750,626000,15350,58400,202000,58500,2512500,33700,37350,81750,6700
AR486291452976955392,Avante Comunicación SL,,"ES, EU",EU-Parliament,67,2200,1950,156750,3850,14600,51000,14500,630000,8400,9375,20250,1700
AR55967615754960896,NEW GEN PUBLISHING SRL,,"EU, RO",EU-Parliament,105,9400,8300,669750,16425,62400,216000,62500,2692500,36100,39975,87750,7200
AR400032566754672640,Fratelli d'Italia Centrodestra Nazionale,,"EU, IT",EU-Parliament,271,61700,54300,4374250,107350,407800,1411000,409500,17557500,235700,260925,571500,46900
AR201649882778304512,AfD-Kreisverband Braunschweig,,"DE, EU",EU-Parliament,15,200,150,13250,325,1200,4000,1000,52500,700,825,1500,150
AR474748779908562944,Future Innovation SRL,,"DE, ES, EU, GB, IT, RO",EU-Parliament,99,164200,144500,11643750,285725,1085400,3757000,1090500,46740000,627300,694575,1522500,124800
AR288284801977286656,Partido Popular Regional De Canarias,,"ES, EU, GB",EU-Parliament,291,5600,4900,393500,9650,36700,127000,37000,1582500,21200,23475,51750,4200
AR474748779908562944,Future Innovation SRL,,"DE, ES, EU, GB, IT, RO",EU-Parliament,99,164200,144500,11643750,285725,1085400,3757000,1090500,46740000,627300,694575,1522500,124800
AR73057599863390208,Liberal Alliance Djursland,,"DK, EU",EU-Parliament,144,5400,4700,380250,9325,35400,123000,35500,1530000,20500,22650,49500,4100


## Advertisers weekly spends

In [26]:

key = 'google-political-ads-advertiser-weekly-spend.csv'
df_1 = bundle_data_1[key]
df_2 = bundle_data_2[key]

assert list(df_1.columns) == list(df_2.columns)

indexes = ['Advertiser_ID', 'Week_Start_Date']
df_1.set_index(indexes, inplace=True)
df_2.set_index(indexes, inplace=True)

ids_1 = set(df_1.index)
ids_2 = set(df_2.index)

assert len(ids_1) == len(df_1)
assert len(ids_2) == len(df_2)

removed_ids = ids_1 - ids_2
added_ids = ids_2 - ids_1
common_ids = ids_1 & ids_2
assert len(ids_1) + len(added_ids) - len(removed_ids) == len(ids_2)
assert len(ids_1 | ids_2) == len(common_ids) + len(added_ids) + len(removed_ids)

print('{} lines were removed, {} were added'.format(len(removed_ids), len(added_ids)))

old_advertisers = set(map(operator.itemgetter(0), removed_ids))
new_advertisers = set(map(operator.itemgetter(0), added_ids))

assert not old_advertisers & new_advertisers

print('Lines have been removed about the following advertisers:\n')
for advertiser_id in old_advertisers:
    print('{} ({})\n'.format(
        'https://transparencyreport.google.com/political-ads/advertiser/{}'.format(advertiser_id),
        [
            removed_id[1]
            for removed_id in removed_ids
            if removed_id[0] == advertiser_id
        ]
    ))

print('Lines have been added about the following advertisers:\n')
for advertiser_id in new_advertisers:
    print('{} ({})\n'.format(
        'https://transparencyreport.google.com/political-ads/advertiser/{}'.format(advertiser_id),
        [
            added_id[1]
            for added_id in added_ids
            if added_id[0] == advertiser_id
        ]
    ))

for line_id in list(common_ids):
    modifications = df_1.loc[line_id] != df_2.loc[line_id]
    modified_columns = list(modifications[modifications].axes[0])
    if modified_columns:
        print('Line {} has be modified'.format(line_id))


72 lines were removed, 251 were added
Lines have been removed about the following advertisers:

https://transparencyreport.google.com/political-ads/advertiser/AR340216110546681856 (['2018-06-24', '2018-10-21', '2018-09-30', '2018-09-09', '2019-04-21', '2018-08-05', '2018-07-01', '2018-09-16', '2018-07-22', '2018-05-27', '2018-07-08', '2018-09-02', '2018-08-26', '2019-04-28', '2018-08-12', '2018-10-14', '2018-06-03', '2018-09-23', '2018-10-28', '2018-08-19', '2018-06-17', '2018-06-10', '2018-07-15', '2018-10-07', '2018-07-29'])

https://transparencyreport.google.com/political-ads/advertiser/AR412273704584609792 (['2018-10-28', '2018-11-04'])

https://transparencyreport.google.com/political-ads/advertiser/AR556388892660465664 (['2019-03-10', '2019-02-24', '2019-02-10', '2019-03-03', '2019-02-17'])

https://transparencyreport.google.com/political-ads/advertiser/AR178086523961344000 (['2018-11-11', '2018-11-04'])

https://transparencyreport.google.com/political-ads/advertiser/AR56539609191

Line ('AR140403236820484096', '2019-05-26') has be modified
Line ('AR6195198266703872', '2019-05-26') has be modified
Line ('AR240608328529477632', '2018-09-23') has be modified
Line ('AR488306308034854912', '2019-05-05') has be modified
Line ('AR383251270535741440', '2019-03-03') has be modified
Line ('AR488306308034854912', '2019-02-10') has be modified
Line ('AR399188966458261504', '2019-05-05') has be modified
Line ('AR485896728302583808', '2019-05-19') has be modified
Line ('AR288284801977286656', '2019-05-19') has be modified
Line ('AR140403236820484096', '2019-04-14') has be modified
Line ('AR474748779908562944', '2019-04-14') has be modified
Line ('AR98551701098725376', '2019-05-26') has be modified
Line ('AR73057599863390208', '2019-05-26') has be modified
Line ('AR227104401595039744', '2019-05-26') has be modified
Line ('AR288179248861020160', '2019-04-07') has be modified
Line ('AR94030784163217408', '2019-03-17') has be modified
Line ('AR373464517536907264', '2019-03-10') h

Line ('AR488306308034854912', '2019-05-26') has be modified
Line ('AR100996740080992256', '2019-06-02') has be modified
Line ('AR474748779908562944', '2019-04-28') has be modified
Line ('AR488306308034854912', '2019-03-03') has be modified
Line ('AR485896728302583808', '2019-04-28') has be modified
Line ('AR230475229367894016', '2018-10-28') has be modified
Line ('AR80178586920681472', '2019-05-19') has be modified
Line ('AR288179248861020160', '2019-03-31') has be modified
Line ('AR75218140211970048', '2019-06-02') has be modified
Line ('AR488306308034854912', '2019-02-03') has be modified
Line ('AR562822410072489984', '2019-06-02') has be modified
Line ('AR198061076825243648', '2019-06-02') has be modified
Line ('AR119011138590474240', '2018-07-15') has be modified
Line ('AR311843487869829120', '2019-06-02') has be modified
Line ('AR532605081762136064', '2019-06-02') has be modified
Line ('AR530356305605427200', '2019-05-12') has be modified
Line ('AR6195198266703872', '2019-04-28') 

## Campaigns

In [27]:
key = 'google-political-ads-campaign-targeting.csv'
df_1 = bundle_data_1[key]
df_2 = bundle_data_2[key]
assert list(df_1.columns) == list(df_2.columns)

duplicate_campaign_ids_1 = [
    campaign_id
    for campaign_id, count in collections.Counter(df_1['Campaign_ID']).items()
    if count > 1
]
duplicate_campaign_ids_2 = [
    campaign_id
    for campaign_id, count in collections.Counter(df_2['Campaign_ID']).items()
    if count > 1
]
assert set(duplicate_campaign_ids_1) == set(duplicate_campaign_ids_2)

print('The following campaigns were run by several advertisers: {}'.format(duplicate_campaign_ids_1))

indexes = ['Campaign_ID', 'Advertiser_ID']
df_1.set_index(indexes, inplace=True)
df_2.set_index(indexes, inplace=True)

ids_1 = set(df_1.index)
ids_2 = set(df_2.index)

assert len(ids_1) == len(df_1)
assert len(ids_2) == len(df_2)

removed_ids = ids_1 - ids_2
added_ids = ids_2 - ids_1
common_ids = ids_1 & ids_2
assert len(ids_1) + len(added_ids) - len(removed_ids) == len(ids_2)
assert len(ids_1 | ids_2) == len(common_ids) + len(added_ids) + len(removed_ids)

print('{} campaigns were removed, {} were added'.format(len(removed_ids), len(added_ids)))

print('Removed campaigns: {}'.format(removed_ids))
# Campaign data can change. For example, End_Date or Gender_Targeting...

The following campaigns were run by several advertisers: ['CA102091303906443264', 'CA12561095713619968', 'CA129676676257808384', 'CA138683050878828544', 'CA168796475340357632', 'CA172438882485272576', 'CA186731709012639744', 'CA190399679802900480', 'CA20351410474319872', 'CA22429487450816512', 'CA239086054680821760', 'CA246748276336885760', 'CA267875667142508544', 'CA271464473095569408', 'CA274551901746364416', 'CA288141865465675776', 'CA288878538256285696', 'CA294897264906731520', 'CA301024568330420224', 'CA304169171585859584', 'CA311790161555881984', 'CA327975797350465536', 'CA328327641071353856', 'CA329242434745663488', 'CA345004208807739392', 'CA346688660621492224', 'CA34819609106317312', 'CA360199459503603712', 'CA360410565736136704', 'CA362455657363800064', 'CA364703059130974208', 'CA364876781968162816', 'CA366002681875005440', 'CA370034316136153088', 'CA373710258385715200', 'CA376226765623787520', 'CA388611664599056384', 'CA392461329685807104', 'CA399779679080284160', 'CA4122555

## Total spending

In [28]:
key = 'google-political-ads-geo-spend.csv'
df_1 = bundle_data_1[key]
df_2 = bundle_data_2[key]

assert list(df_1.columns) == list(df_2.columns)

indexes = ['Country', 'Country_Subdivision_Primary', 'Country_Subdivision_Secondary']
df_1.set_index(indexes, inplace=True)
df_2.set_index(indexes, inplace=True)
ids_1 = set(df_1.index)
ids_2 = set(df_2.index)

assert len(ids_1) == len(df_1) == len(ids_2) == len(df_2)

removed_ids = ids_1 - ids_2
added_ids = ids_2 - ids_1
common_ids = ids_1 & ids_2
assert len(removed_ids) == 0
assert len(added_ids) == 0

for line_id in list(common_ids):
    suspicious_modifications = df_1.loc[line_id].map(int) > df_2.loc[line_id].map(int)
    modified_columns = list(suspicious_modifications[suspicious_modifications].axes[0])
    if modified_columns:
        print('Line {} has be modified'.format(line_id))

Line ('US', 'US-PA', 'PA-14') has be modified
Line ('US', 'US-PA', 'PA-11') has be modified
Line ('US', 'US-PA', 'PA-2') has be modified
Line ('US', 'US-PA', 'PA-13') has be modified
Line ('PL', '', '') has be modified
Line ('US', 'US-PA', 'PA-15') has be modified
Line ('US', 'US-NC', 'NC-4') has be modified


In [29]:
line_id = ('PL', '', '')
df_1.loc[line_id]['Spend_EUR'], df_2.loc[line_id]['Spend_EUR']

('201300', '199200')